In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install jpype1
!pip install konlpy
!pip install git+https://github.com/lovit/textrank.git

     |████████████████████████████████| 448 kB 5.1 MB/s 
     |████████████████████████████████| 19.4 MB 4.4 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Cloning https://github.com/lovit/textrank.git to /tmp/pip-req-build-wxmjl7pp
  Running command git clone -q https://github.com/lovit/textrank.git /tmp/pip-req-build-wxmjl7pp
  Created wheel for textrank: filename=textrank-0.1.2-py3-none-any.whl size=11171 sha256=3cad2275d47561e6a5b85e2dd5f382e266a4c8236c63c6edeae780a54b412e26
  Stored in directory: /tmp/pip-ephem-wheel-cache-iolw5xkl/wheels/92/77/3d/d596fd63962c68b352c2e39170c2cbe21b9fca35e89aafdf87
Successfully built textrank


In [3]:
!pip install transformers==2.2.0
!pip install spacy==2.0.12

     |████████████████████████████████| 360 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 47.4 MB/s 
     |████████████████████████████████| 131 kB 61.9 MB/s 
     |████████████████████████████████| 895 kB 23.9 MB/s 
     |████████████████████████████████| 8.5 MB 24.6 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 138 kB 62.1 MB/s 
     |████████████████████████████████| 127 kB 61.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 22.0 MB 1.3 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 

In [4]:
!pip install bert-extractive-summarizer==0.4.2 # in colab
# !pip install bert-extractive-summarizer

     |████████████████████████████████| 125 kB 5.8 MB/s 
  Created wheel for bert-extractive-summarizer: filename=bert_extractive_summarizer-0.4.2-py3-none-any.whl size=13710 sha256=c8e78348ec8796d69c276e3e0a814fdf5e20c9a68619b1acce3761d10634c968
  Stored in directory: /root/.cache/pip/wheels/72/dc/37/8f4d3e5a337d48b296ebe21196d99dd5a52d494411b5ff9825
Successfully built bert-extractive-summarizer
  Attempting uninstall: cymem
    Found existing installation: cymem 1.31.2
    Uninstalling cymem-1.31.2:
      Successfully uninstalled cymem-1.31.2
  Attempting uninstall: preshed
    Found existing installation: preshed 1.0.1
    Uninstalling preshed-1.0.1:
      Successfully uninstalled preshed-1.0.1


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Dec 28 23:56:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [7]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from konlpy.tag import Komoran
from textrank import KeysentenceSummarizer
from summarizer import Summarizer, TransformerSummarizer

In [8]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [9]:
BASE_DIR = "/content/drive/MyDrive/AI 부트캠프/Section Project/Codestates Project 1/data"
TRAIN_DIR = os.path.join(BASE_DIR, "train/")
VALID_DIR = os.path.join(BASE_DIR, "valid/")

In [10]:
os.listdir(TRAIN_DIR), os.listdir(VALID_DIR)

(['newspaper_train.json', 'law_train.json', 'article_train.json'],
 ['newspaper_valid.json', 'article_valid.json', 'law_valid.json'])

In [11]:
with open(TRAIN_DIR + "newspaper_train.json", "r", encoding="utf-8") as f:
    news_train = json.load(f)

for json_str in news_train:
    print(json_str)

name
delivery_date
documents


In [12]:
with open(VALID_DIR + "newspaper_valid.json", "r", encoding="utf-8") as f:
    news_valid = json.load(f)

for json_str in news_valid:
    print(json_str)

name
delivery_date
documents


In [13]:
def prepro_text(json_data, n):
    one = []
    item = json_data["documents"][n]

    one.append(item["category"])
    one.append(item["media_name"])
    one.append(item["media_type"])
    one.append(item["char_count"])

    full_temp = []
    for i in range(len(item["text"])):
        if item["text"][i]:
            for j in range(len(item["text"][i])):
                full_temp.append(item["text"][i][j]["sentence"])

    one.append(full_temp)
    one.append(" ".join([str(n) for n in item["extractive"]]))

    extract_temp = []
    for i in item["extractive"]:
        extract_temp.append(full_temp[i])
    
    one.append(" ".join(extract_temp))
    one.append(item["abstractive"][0])

    return one

In [14]:
def json_to_df(json_data):
    df = pd.DataFrame(columns = ["category", "media_name", "media_type", "char_count", "full_text", "extractive_idx", "extractive", "abstractive"])

    #for i in tqdm(range(100)):
    for i in tqdm(range(len(json_data["documents"]))):
        try:
            df.loc[i] = prepro_text(json_data, i)
        except:
            print(i)
            pass

    return df

In [15]:
train_df = json_to_df(news_train)

  0%|          | 0/243983 [00:00<?, ?it/s]

21810
50279
61894
67953
68553
184704


In [16]:
train_df.head()

,category,media_name,media_type,char_count,full_text,extractive_idx,extractive,abstractive
0,종합,광양신문,online,927,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",2 3 10,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,종합,광양신문,online,764,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",2 4 11,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,스포츠,광양신문,online,1066,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",3 5 7,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,정치,광양신문,online,746,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...",2 3 4,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,종합,광양신문,online,978,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",3 7 4,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [17]:
train_df.reset_index(drop=True, inplace=True)

In [18]:
train_df.to_csv(BASE_DIR + "/prepro_train_all2.tsv", sep="\t", index=False)

In [19]:
valid_df = json_to_df(news_valid)

  0%|          | 0/30122 [00:00<?, ?it/s]

4318


In [20]:
valid_df.head()

,category,media_name,media_type,char_count,full_text,extractive_idx,extractive,abstractive
0,정치,한국경제,online,1111,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",0 1 3,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,종합,한국경제,online,904,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",2 1 6,바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧했...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,"IT,과학",한국경제,online,1104,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",1 5 8,지난 3일 한국이 세계 최초로 5세대(5G) 이동통신 서비스를 상용화한 것을 기념하...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,사회,한국경제,online,740,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,0 1 2,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,경제,한국경제,online,1389,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",1 3 4,기업 지배구조 개선 등을 통해 높은 수익률을 올리는 것을 목표로 하는 공모펀드 'K...,주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [21]:
valid_df.reset_index(drop=True, inplace=True)

In [22]:
valid_df.to_csv(BASE_DIR + "/prepro_valid_all2.tsv", sep="\t", index=False)

## **TextRank**

In [23]:
komoran = Komoran()

def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

textrank_summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)

In [24]:
textrank_sum = []
for i in tqdm(range(len(train_df))):
    text = train_df.loc[i, "full_text"]
    keysents = textrank_summarizer.summarize(text, topk=3)
    textrank_sum.append(keysents)

  0%|          | 0/243977 [00:00<?, ?it/s]

ValueError: ignored

In [31]:
textrank_sum2 = []
for i in tqdm(range(175798,len(train_df))):
    try:
    
        text = train_df.loc[i, "full_text"]
        keysents = textrank_summarizer.summarize(text, topk=3)
        textrank_sum2.append(keysents)
    except:
        print(i)
        pass

  0%|          | 0/68179 [00:00<?, ?it/s]

175798
177913
182816
183013


In [36]:
train_df.loc[175798,"full_text"]

['SINo ALICE is a fairy tale-themed smartphone video game for iOS and Android devices launched first in Japan in June 2017. Backed by cult-like followers, the game tops the mobile app rankings by revenue in Japan.',
 'Nexon acquired the global publishing rights for the dark fantasy RPG last year in the areas except for Japan, China, Hong Kong, Taiwan and Macau, where the label is already available. It was part of efforts to enhance its grip in the mobile game market with new titles, on top of fresh investment in foreign companies.',
 'Nexon separately announced Monday it has made a strategic investment in Stockholm-based Embark Studios to buy a 32.8 percent stake to hold total 66.1 percent in the Swedish game company. Embark Studios was founded last year by Patrick Soderlund, former chief design officer of Electronic Arts and chief executive officer of DICE, famous for the first-person shooter video game series Battlefield.']

In [37]:
train_df.loc[177913,"full_text"]

['Weak global oil prices have driven down costs for producers and service providers, the central bank said.',
 "The price of Dubai crude, the country's benchmark that makes up nearly 85 percent of total oil imports, averaged $61.78 per barrel in June, down $7.6 from a month ago.",
 'Prices of factory produces declined 0.6 percent from a month earlier. Coal and petroleum products prices lost 6.5 percent on month to drive down the overall PPI. Gasoline prices were off 10.8 percent, and naphtha 14.1 percent. Chemical products prices were down 0.4 percent, and computers/electrical and optical devices prices also 0.4 percent. DRAM prices contracted 5.3 percent on increased inventory levels from poor demand.',
 'Prices of agricultural, livestock and fishery products dropped 0.5 percent on increased crop yield. Potato prices plunged 47.5 percent, watermelon 17.1 percent, and Korean melon 18.8 percent.']

In [38]:
train_df.loc[182816,"full_text"]

['"Most of Korea\'s shipments to Hong Kong are rechanneled to China so escalating protests would further dampen exports to China especially in semiconductors," the report said.',
 "Korea sold $46 billion worth of goods to Hong Kong last year, making it the country's fourth-largest export destination after China, the United States and Vietnam. Semiconductors were the dominant export item, accounting for 73 percent of all Hong Kong-bound shipments, with memory chips making up 63.3 percent. Computers followed with 3.4 percent, cosmetics 2.9 percent, petroleum products 2.7 percent, and petrochemical products 2.4 percent.",
 "For Korea, Hong Kong is an important trading route to mainland China. Nearly 82.6 percent, or $29.3 billion of Korea's total $35.5 billion worth of shipments to Hong Kong were exported to China last year.",
 "Korea has so far taken advantage of Hong Kong's status as a global financial hub, benefiting from its low taxes, zero tariffs and stable legal framework.",
 'If t

In [ ]:
train_df.drop([175798,177913,182816,183013], inplace=True)

In [42]:
train_df.reset_index(drop=True, inplace=True)

In [25]:
len(textrank_sum)

175798

In [27]:
textrank_sum[175797]

[(13,
  1.6429414738813595,
  '시몬스 침대 관계자는 "이번 조사 결과를 통해 응답자 대부분이 침실 내 화재 위험성과 피해현황에 잘 모르고 있다는 점이 드러나, 실제 화재 관련 통계와 국민들의 인식 수준 사이에 큰 차이가 있는 것을 확인할 수 있었다"며 "한국 시몬스가 2018년 11월 출시한 불에 잘 타지 않는 난연 매트리스는 화재 시 재실자의 피난은 물론 소방관들의 화재 진압 활동시 안전 확보에도 도움을 줄 수 있다"고 말했다.'),
 (2,
  1.2132977533837792,
  '조사 결과 최근 3년간 생활공간에서 발생한 화재 사고 중 침실에서 가장 많은 사망자가 발생했다는 사실을 알고 있는 응답자는 불과 10.8%에 그쳤다.'),
 (10,
  1.1734949336362188,
  '이 중 침대 매트리스는 부피가 크고 겉감이 섬유로 싸여있으며 각종 고분자 화합물이 포함돼 있어 응답자들의 인식과 같이 침실 화재 시 플래시오버(화재가 급격히 확산되는 단계)를 일으키는 주요 매개체로 꼽히고 있다.')]

In [43]:
textrank_res = []
for n in range(len(textrank_sum)):
    result = textrank_sum[n][0][2] + "\n" + textrank_sum[n][1][2] + "\n" + textrank_sum[n][2][2]
    textrank_res.append(result)
for n in range(len(textrank_sum2)):
    result = textrank_sum2[n][0][2] + "\n" + textrank_sum2[n][1][2] + "\n" + textrank_sum2[n][2][2]
    textrank_res.append(result)

In [46]:
len(textrank_res), len(train_df)

(243973, 243973)

In [47]:
train_df["textrank_summary"] = textrank_res
train_df.head()

,category,media_name,media_type,char_count,full_text,extractive_idx,extractive,abstractive,textrank_summary
0,종합,광양신문,online,927,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",2 3 10,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼...
1,종합,광양신문,online,764,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",2 4 11,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,시 관계자는“컬러빌리지 사업은 주민들의 적극적인 관심과 참여가 만들어낸 값진 성과물...
2,스포츠,광양신문,online,1066,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",3 5 7,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,전남은 이어 측면 공격력 강화를 위해 포항에서 활약한 완델손을 1년 임대 영입했다....
3,정치,광양신문,online,746,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...",2 3 4,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","교육은 시의 특화작목인 감, 참다래, 생강, 매실 등에 대한 저비용 생산과 고품질 ..."
4,종합,광양신문,online,978,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",3 7 4,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"전남도 관계자는“중국 중심의 크루즈 유치에서 일본, 타이완 등 시장 다변화 노력의 ..."


## **bert extractive summarization**

In [48]:
bert_model = Summarizer()
bert_temp = []
for i in tqdm(range(len(train_df))):
    try:
        bert_text = train_df.loc[i,"full_text"]
        bert_summary = ''.join(bert_model(" ".join(bert_text), min_length=60))
        bert_temp.append(bert_summary)
    except:
        bert_temp.append(np.nan)
        pass

100%|██████████| 231508/231508 [00:00<00:00, 3169832.96B/s]


  0%|          | 0/243973 [00:00<?, ?it/s]

In [49]:
train_df["bert_summary"] = bert_temp

In [50]:
train_df["bert_summary"].isnull().sum()

243795

In [51]:
train_df

,category,media_name,media_type,char_count,full_text,extractive_idx,extractive,abstractive,textrank_summary,bert_summary
0,종합,광양신문,online,927,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",2 3 10,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼...,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...
1,종합,광양신문,online,764,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",2 4 11,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,시 관계자는“컬러빌리지 사업은 주민들의 적극적인 관심과 참여가 만들어낸 값진 성과물...,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n..."
2,스포츠,광양신문,online,1066,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",3 5 7,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,전남은 이어 측면 공격력 강화를 위해 포항에서 활약한 완델손을 1년 임대 영입했다....,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...
3,정치,광양신문,online,746,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...",2 3 4,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","교육은 시의 특화작목인 감, 참다래, 생강, 매실 등에 대한 저비용 생산과 고품질 ...","11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n..."
4,종합,광양신문,online,978,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",3 7 4,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"전남도 관계자는“중국 중심의 크루즈 유치에서 일본, 타이완 등 시장 다변화 노력의 ...","홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma..."
...,...,...,...,...,...,...,...,...,...,...
243968,종합,중도일보,online,751,"[내달 12일까지, 노동인권 감수성 높인다, 당진시는 지난 12일 첫 교육을 시작으...",1 2 6,당진시는 지난 12일 첫 교육을 시작으로 내달 12일까지 당진지역 청소년들을 대상으...,당진시는 12일부터 당진시 노동상담소에서지역 청소년들을 대상으로 노동 인권 교육을 ...,"시 관계자는 ""청소년 노동자의 권리 보호를 위해 찾아가는 노동인권 교육을 실시해 청...",NaN
243969,종합,중도일보,online,858,"[마을별 10명 이상이면 신청가능, 12월 24일까지 모집, 서산 마을학교 합동졸업...",5 4 8,"성인문해교실(마을학교)은 학령기 동안 경제적, 사회적 환경 등의 어려움으로 배움의 ...",서산시는 배움의 기회를 갖지 못한 어르신을 대상으로 성인문해교실(마을학교)를 운영하...,서산시(시장 맹정호)가 2020년 신규 성인문해교실(마을학교) 개강을 희망하는 마을...,NaN
243970,종합,중도일보,online,882,"[2020년 예산안, 제4회 추경예산안 및 조례안 29건 등 41개 안건 처리 예정...",1 3 4,서산시의회는 25일 제1차 본회의를 열고 제247회 제2차 정례회를 개회했다. 다음...,서산시의회는 25일 제1차 본회의를 열고 다음달 13일까지 열리는 제247회 제2차...,다음달 13일까지 총 19일간 열리는 이번 정례회에서는 2020년도 예산안과 제4회...,NaN
243971,종합,중도일보,online,938,"[전담기관 설립 추진, 타당성 검토 용역 착수보고회 개최, 아산시(시장 오세현)가 ...",1 2 3,아산시(시장 오세현)가 온천수를 활용한 재활 헬스케어 힐링산업에 본격 착수할 예정이...,아산시가 25일 시청 상황실에서 전국적 허브를 구축하기 위한 재활헬스케어힐링스파산업...,시는 지난 5월 산업부 스마트특성화기반구축사업 적합사업으로 선정된 '힐링스파 기반 ...,NaN
